#First, load in "single_clean_day.csv" ride data, and add time windows. Save output.

In [18]:
import numpy as np
import pandas as pd
import itertools
import operator
from haversine import haversine
import time

#set cd to data folder with QC'ed rides
% cd /Users/fineiskid/Desktop/DSSG_ffineis/main_repo/Access_Analysis_Rproject/data/


/Users/fineiskid/Desktop/DSSG_ffineis/main_repo/Access_Analysis_Rproject/data


In [19]:
data = pd.read_csv("single_clean_day.csv")

In [20]:
data["ServiceDate"] = pd.to_datetime(data["ServiceDate"], format ='%Y-%m-%d %H:%M:%S', unit = "D")

In [21]:
def add_TimeWindows(data, windowsz):
    '''calculate time windows (pickup and dropoff)
        from SchTime and ETA.
        data is subsetted schedule data from a day.
        windowsz is size of pickup/dropoff window in seconds'''
    
    etas = data.loc[:,"ETA"]
    schtime =data.loc[:,"SchTime"]
    schtime.loc[np.where(schtime<0)] = np.nan
    schtime.head()
    windowsz = 60*30
    sLength = data.shape[0]
    data.insert(len(data.columns), 'PickupStart',  pd.Series(np.zeros(sLength), index=data.index))
    data.insert(len(data.columns), 'PickupEnd',  pd.Series(np.zeros(sLength), index=data.index))
    data.insert(len(data.columns), 'DropoffStart',  pd.Series(np.zeros(sLength), index=data.index))
    data.insert(len(data.columns), 'DropoffEnd',  pd.Series(np.zeros(sLength), index=data.index))
    data.head(10)
    for x in range(0, sLength):

        #make dropoff window when there's no required drop off time
        if (data.Activity.loc[x] == 1) & (data.ReqLate.loc[x] <0):
            data.DropoffStart.loc[x] = data.ETA.loc[x]-3600
            data.DropoffEnd.loc[x] = data.ETA.loc[x]+3600

        #make dropoff window when there IS a required drop off time: 1hr before ReqLate time
        if (data.loc[x, "Activity"] == 1) & (data.loc[x, "ReqLate"] >0):
            data.loc[x,"DropoffStart"] = data.loc[x, "ETA"]-3600
            data.loc[x, "DropoffEnd"] = data.loc[x, "ReqLate"]

        #schtime is in the middle of the pick up window
        if data.loc[x, "Activity"] == 0:
            data.loc[x, "PickupStart"] = schtime.loc[x]-(windowsz/2)
            data.loc[x, "PickupEnd"] = schtime.loc[x]+(windowsz/2)

    return data

In [ ]:
data = add_TimeWindows(data, 60*30)
data.to_csv(path = "/Users/fineiskid/Desktop/DSSG_ffineis/main_repo/Access_Analysis_Rproject/data/single_day_TimeWindows.csv", index = False, quoting=None)

#If you've already created the "single_clean_day.csv" file with appended time windows, skip step 1 and jump here:



In [55]:
try:
    data = pd.read_csv('single_day_TimeWindows.csv')
except IOError:
    print "File does not exist."


In [56]:
def get_URIDs(data, broken_Run, resched_init_time):
    '''get unscheduled request id's from broken bus,
        based on when we're allowed to first start rescheduling.
        resched_init_time is in seconds, marks the point in time we can begin considering reinserting new requests.
        broken_Run is number of run that breaks
        data is today's scheduling data

        RETURN: list of URIDs'''
    
    #all rides that exist past time we're allowed to begin rescheduling
    leftover = data[data["ETA"] >= resched_init_time]
    leftover = leftover[(leftover["Activity"] != 6) & (leftover["Activity"] != 16) & (leftover["Activity"] != 3)]
    
    #rides that were scheduled to be on broken bus past resched_init_time
    pickmeup = leftover[leftover["Run"]==broken_Run]
    clients = pickmeup["ClientId"].unique()
    clients = clients[~(np.isnan(clients))]
    unsched = pickmeup

    print("There are %s rides left to be scheduled on broken run %s" % (unsched.shape[0], broken_Run))

    class URID:
        def __init__(self, BookingId, Run, PickUpCoords, DropOffCoords, PickupStart, PickupEnd, DropoffStart, DropoffEnd, SpaceOn, MobAids):
            self.BookingId= BookingId
            self.Run = Run
            self.PickUpCoords = PickUpCoords
            self.DropOffCoords = DropOffCoords
            self.PickupStart = PickupStart
            self.PickupEnd = PickupEnd
            self.DropoffStart = DropoffStart
            self.DropoffEnd = DropoffEnd
            self.SpaceOn = SpaceOn
            self.MobAids = MobAids

    diffIDs = unsched.BookingId.unique()
    saveme = []

    #save separate URID's in a list
    for ID in diffIDs:
        my_info = unsched[unsched["BookingId"]==ID]
        #if person is already on bus when breakdown occurs,
        #need to handle URID differently:
        if(my_info.shape[0] == 1):
            temp = URID(BookingId = ID,
                Run = broken_Run,
                PickUpCoords = pd.Series(data = np.array(BREAKDOWN_LOC), index = ["LAT", "LON"]),
                DropOffCoords = my_info[["LAT", "LON"]].iloc[0,],
                PickupStart = resched_init_time,
                PickupEnd = resched_init_time+30*60,
                DropoffStart = int(my_info[["DropoffStart"]].iloc[0,]),
                DropoffEnd = int(my_info[["DropoffEnd"]].iloc[0,]),
                SpaceOn = my_info[["SpaceOn"]].iloc[0,],
                MobAids = my_info[["MobAids"]].iloc[0,])
        else:
            temp = URID(BookingId = ID,
                Run = broken_Run,
                PickUpCoords = my_info[["LAT", "LON"]].iloc[0,],
                DropOffCoords = my_info[["LAT", "LON"]].iloc[1,],
                PickupStart = int(my_info[["PickupStart"]].iloc[0,]),
                PickupEnd = int(my_info[["PickupEnd"]].iloc[0,]),
                DropoffStart = int(my_info[["DropoffStart"]].iloc[1,]),
                DropoffEnd = int(my_info[["DropoffEnd"]].iloc[1,]),
                SpaceOn = my_info[["SpaceOn"]].iloc[0,],
                MobAids = my_info[["MobAids"]].iloc[0,])
        
        saveme.append(temp)

    return saveme

In [57]:
#Global Variables: broken_Run, resched_init_time, BREAKDOWN_LOC
broken_Run = data.Run.unique()[0]
resched_init_time = 14*60*60
BREAKDOWN_LOC = [47.661254, -122.330299]
urids = get_URIDs(data, broken_Run, resched_init_time)

There are 12 rides left to be scheduled on broken run 680SEB


In [58]:
def get_busRuns(data, Run, resched_init_time):
    ''' take pd.DataFrame from add_Time_Windows.py and create busRun object for specified Run number,
    for all stops occurring at or after resched_init_time.

    RETURN: busRun object for specified Run.'''

    #subset based on matching Run number, and subset for stops only at or after resched_init_time
    print("Getting remaing rides for run %s" % Run)
    dataSub = data[(data["Run"] == Run)] # & (data["ETA"] >= resched_init_time)]
    #subset only the rides that aren't 6, 16, or 3:
    leave = dataSub.index.min()
    gas = dataSub[(dataSub["Activity"]==6)|(dataSub["Activity"]==16)|(dataSub["Activity"]==3)].index.min()
    busRun = dataSub.loc[leave:gas]

    return busRun

In [59]:
#list urid attributes:
URID = urids[0]
int(URID.DropoffStart)

46897

In [60]:
#Find windows that overlap 
def time_overlap(Run_Schedule, URID, pickUpDropOff = True):
    '''URID: of class URID, has bookingId, pickUpLocation, dropOffLocation, etc.
        Run_Schedule: Schedule (pd.Data.Frame) of the run on which we're trying to insert the URID
        RETURN: dictionary containing indices of schedule-outbound and -inbound nodes that we need
        to get distance between w/r/t URID location.'''

    #How it works: first, find all nodes that have time overlap with the URID's (pickup or dropoff) window
    # second, notice any gaps in the order of these nodes from the original bus ride.
    # For the first chunk of overlapping nodes, add the lower-time bound node in, given that the first node
    # even exists in the Run_Schedule.
    # For every chunk of contiguous overlap nodes, add in the upper-time bound node, because there can potentially
    # be an inbound ride from the URID node to the upper-time bound node. There can't be an outbound ride.

    if pickUpDropOff:
        Start = URID.PickupStart
        End = URID.PickupEnd
    else:
        Start = URID.DropoffStart
        End = URID.DropoffEnd
        
    crossover = []
    
    for jj in range(Run_Schedule.shape[0]):
        #Checking if a Run's PickupWindow overlaps with URID's Window.
        if Run_Schedule.Activity.iloc[jj] == 0:
            PUE = Run_Schedule.PickupEnd.iloc[jj]; PUS = Run_Schedule.PickupStart.iloc[jj]
            #simple, unequal overlap
            if (PUE > Start) & (PUS < End):
                crossover.append(Run_Schedule.index[jj])
            # equal or strictly within [PUS, PUE]
            if (PUE <= End) & (PUS >= Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered by [PUS, PUE] and then some on both sides
            if (PUE > End) & (PUS < Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered and then some only on left side
            if (PUE == End) & (PUS < Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered and then some only on right side
            if (PUS == Start) & (PUE > End):
                crossover.append(Run_Schedule.index[jj])
                
        #Checking if a Run's DropoffWindow overlaps with URID's Window.
        if Run_Schedule.Activity.iloc[jj] == 1:
            DOE = Run_Schedule.DropoffEnd.iloc[jj]; DOS = Run_Schedule.DropoffStart.iloc[jj]
            if (DOE > Start) & (DOS < End):
                crossover.append(Run_Schedule.index[jj])
            if (DOE <= End) & (DOS >= Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOE > End) & (DOS < Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOE == End) & (DOS < Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOS == Start) & (DOE > End):
                crossover.append(Run_Schedule.index[jj])
                
    #Get rid of cases that repeat themselves:
    crossover = list(set(crossover))
    
    inserts = Run_Schedule.loc[crossover]; indices = Run_Schedule.index
    lst = []; outbound = []; inbound = []
    #Lists of continuously arranged nodes with overlap
    for k, g in itertools.groupby(enumerate(inserts.index), lambda (i,x):i-x):
        k = map(operator.itemgetter(1), g)
        #save upper/lower bound where appropriate
        lst += [min(k)]; lst+=[max(k)]
        if len(lst) == 2:
            outbound += k; inbound +=k
            if lst[0] != min(indices):
                outbound.append(lst[0]-1) #we have a lower bound node, heading outbound from Run_Schedule
            else:
                inbound.pop(0) #if first node is leave garage, can't add lower bound
        else:
            outbound += k
            inbound +=k[1:len(k)]
        
        inbound.append(lst[-1]+1) #upper bound node for first contiguous set of overlap nodes
    
    outbound, inbound = map(sorted, [outbound, inbound])
    all_nodes = sorted(np.union1d(outbound, inbound))
    print("Need to service URID within %s sec to %s sec" % (Start, End))
    print("...%s nodes fall within this criteria." % len(all_nodes))
    #print("These indices of Run_Schedule will need to have distances calculated:\n%s" % np.union1d(outbound, inbound))

    retDict = {"outbound": outbound, "inbound": inbound, "all_nodes" : all_nodes}
    return retDict

In [61]:
#print distances that bus 181SEB is from URID location during time windows... It's pretty far...
def radius_Elimination(data, URID, radius, pickUpDropOff):
    '''Given a set of the day's bus data and an unhandled requst,
    eliminate all bus routes that are farther than radius-miles away at all
        points in the URID's pickup or dropoff window, the window being specified
        by pickUpDropOff. Run time_insertions.py on the resultant list.

        NOTE: you need to "> pip install haversine"

        INPUT:  data - pd.Data.Frame returned from add_TimeWindows.py
                URID - URID object from get_URIDS.py	
                radius - float, number of miles
                pickUpDropOff - boolean True/False for PickUp (True) or Dropoff (False)

        RETURN: LIST of runs within radius-miles of URID.'''

    #obviously, broken bus can't be in the list of nearby buses.
    data = data[data.Run != URID.Run]

    if pickUpDropOff:
        URID_loc = ([URID.PickUpCoords["LAT"], URID.PickUpCoords["LON"]])
    else:
        URID_loc = ([URID.DropOffCoords["LAT"], URID.PickUpCoords["LON"]])
        
    #get pd.Data.Frame of nodes that have overlap with URID's pickup or dropoff window
    overlap_data = time_overlap(data, URID, pickUpDropOff)

    #get row index of nodes that may have either inbound/outbound overlap with URID TW.
    overlap_data = data.loc[overlap_data['all_nodes']]
    overlap_LAT = overlap_data.LAT.tolist()
    overlap_LON = overlap_data.LON.tolist()

    #store list of rides that are sufficiently nearby URID's location
    okBuses = []
    for k in range(len(overlap_LAT)):
        point = (overlap_LAT[k], overlap_LON[k])
        dist = haversine(point, URID_loc, miles=True)
        if(dist < radius):
            okBuses.append(overlap_data.Run.iloc[k])

    return list(set(okBuses))

In [62]:
radius =  4#miles cutoff

#list of buses to plug URID onto:
good_buses = radius_Elimination(data, URID, radius, True)
print("At 1:30 PM, there are %s buses within %s miles that have nodes that overlap with URID." %
      (len(good_buses), radius))

#Test get_busRuns subsetting routine
testRun = good_buses[20]
#Bus 335: try inserting URID on to here.
Run_Schedule = get_busRuns(data, testRun, resched_init_time)

Need to service URID within 50400 sec to 52200 sec
...1106 nodes fall within this criteria.
At 1:30 PM, there are 53 buses within 4 miles that have nodes that overlap with URID.
Getting remaing rides for run 502R


In [63]:
#Get distances between (insertable nodes, URID location)
import requests
def osrm (URID_location, inbound, outbound):
    #URID_location it's a list: [lat, lon]
    #lists for inbound and outbound matrices
    # inbound/outbound: 2-column np.arrays storing inbound/outbound node latitude/longitude
    # and inbound (from scheduled location to urid location) 
    out_total_time = []
    out_start_points = []
    out_end_points = []
    in_total_time = []
    in_start_points = []
    in_end_points = []
    out_osrm_url = "http://router.project-osrm.org/viaroute?"
    in_osrm_url = "http://router.project-osrm.org/viaroute?"
    urid_LAT = URID_location[0]; urid_LON = URID_location[1]

    # outbound
    for lat_cord,lon_cord in outbound: 
        out_route_url = out_osrm_url+ "loc=" + str(lat_cord) + "," + str(lon_cord)
        out_route_url = out_route_url + "&loc=" + str(urid_LAT) + "," + str(urid_LON) + "&instructions=false"
        out_route_requests = requests.get(out_route_url)
        out_route_results = out_route_requests.json()
        out_total_time += [out_route_results[u'route_summary'][u'total_time']]
        out_start_points += [out_route_results[u'route_summary'][u'start_point']]
        out_end_points += [out_route_results[u'route_summary'][u'end_point']]

    # inbound
    for lat_cord,lon_cord in inbound: 
        in_route_url = in_osrm_url + "loc=" + str(urid_LAT) + "," + str(urid_LON) 
        in_route_url = in_route_url+ "&loc=" + str(lat_cord) + "," + str(lon_cord) + "&instructions=false"
        in_route_requests = requests.get(in_route_url)
        in_route_results = in_route_requests.json()
        in_total_time += [in_route_results[u'route_summary'][u'total_time']]
        in_start_points += [in_route_results[u'route_summary'][u'start_point']]
        in_end_points += [in_route_results[u'route_summary'][u'end_point']]

    a = np.array([out_total_time])
    b = np.array([in_total_time])
 
    return(np.hstack((a.T, b.T)))


In [64]:
#Time the travel time matrix function:

#formulate URID's location:
uridLoc = [URID.PickUpCoords[0], URID.PickUpCoords[1]]

#Get nodes from 181SEB bus that overlap with URID
pickUpDropOff = True; URID = urids[0];

#formulate inbound nodes and outbound nodes, for input into OSRM!!!
inserts = time_overlap(Run_Schedule, URID, pickUpDropOff)
outbound = Run_Schedule.loc[inserts["outbound"]]
outbound = np.column_stack((np.array(outbound.LAT), np.array(outbound.LON))) #GODDAMNIT THAT WAS COMPLICATED
inbound = Run_Schedule.loc[inserts["inbound"]]
inbound = np.column_stack((np.array(inbound.LAT), np.array(inbound.LON)))

#print("inbound nodes: %s" % inbound)
#print("outbound nodes: %s" % outbound)

start = time.clock()
time_matrix = osrm(uridLoc, inbound, outbound)
elapsed = time.clock()-start
print("API call took %s sec" % elapsed)

Need to service URID within 50400 sec to 52200 sec
...6 nodes fall within this criteria.
API call took 0.039586 sec


In [65]:
#We now have travel times for the inbound, outbound insertions!
print(time_matrix)
Run_Schedule.loc[inserts["all_nodes"]]

[[658 831]
 [826 164]
 [192 201]
 [200 408]
 [399 476]]


,ServiceDate,Run,ProviderId,EvOrder,EvId,ReqTime,SchTime,ReqLate,Activity,ETA,...,PassOn,SpaceOn,PassOff,SpaceOff,ClientId,MobAids,PickupStart,PickupEnd,DropoffStart,DropoffEnd
512,2014-04-14,502R,5,49818844,11861,-1,45900,-1,7,49200,...,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0
513,2014-04-14,502R,5,52131175,16876,51480,51480,-1,0,51190,...,CLI1,AM1,NaN,NaN,124111,LI,50580,52380,0,0
514,2014-04-14,502R,5,53673356,13689,52500,52500,-1,0,52904,...,CLI1,WX1,NaN,NaN,92953,AP,51600,53400,0,0
515,2014-04-14,502R,5,54187411,16877,-1,NaN,55800,1,53573,...,NaN,NaN,CLI1,AM1,124111,LI,0,0,49973,55800
516,2014-04-14,502R,5,56136252,13679,-1,NaN,-1,1,54606,...,NaN,NaN,CLI1,WX1,92953,AP,0,0,51006,58206
517,2014-04-14,502R,5,61475448,15017,51300,54900,-1,0,56888,...,CLI1,WG1,NaN,NaN,32104,WA,54000,55800,0,0


###Cool, now we have everything we need to start checking (pick-up) feasibility:
1. We have ``good_buses``: list of buses within radius that have stops overlapping with the URID's time window. Don't worry, the broken bus isn't on this list.

2. We have ``Run_Schedule``: a Pandas dataframe that has all stops relevant to squeezing in URID.

3. We have ``inserts``: dictionary with insertion nodes on the scheduled buses that have overlapping time windows.

4. We have ``outbound_times`` and ``inbound_times``: matrices travel times between the chosen scheduled bus's insertion nodes and the URID's location.


###Feasibility checking: insert URID into run, find shortest additional time added. Also, find number of time windows broken by each insertion.

######Also: remember that len(inbound) and len(outbound) should be equal!

In [66]:
# FEASIBILITY OF PICK UP:

#location from where we'll pick up given URID.
uridLoc = [URID.PickUpCoords[0], URID.PickUpCoords[1]]

pickup_inserts = time_overlap(Run_Schedule, URID, pickUpDropOff = True)
outbound = Run_Schedule.loc[pickup_inserts["outbound"]]
outbound = np.column_stack((np.array(outbound.LAT), np.array(outbound.LON)))
inbound = Run_Schedule.loc[pickup_inserts["inbound"]]
inbound = np.column_stack((np.array(inbound.LAT), np.array(inbound.LON)))

time_matrix_pickup = osrm(uridLoc, inbound, outbound)

#start picking best pickup insertion:
rt_times = sorted(enumerate(np.sum(time_matrix_pickup, 1)), key=operator.itemgetter(1)) #use itemgetter(1) because (0) is index from enumerator!

#smallest round trip travel time, corresponding rows on bus's schedule:
best_rt_time = rt_times[0][1]
#row indices on Run_Schedule between which to insert:
leave1 = pickup_inserts["outbound"][rt_times[0][0]] #leave this scheduled node
comeback1 = pickup_inserts["inbound"][rt_times[0][0]] #come back to this scheduled node

dwell = 500

#get total lag time, see if next time window is broken:
newETA = Run_Schedule.ETA.loc[leave1] + dwell + best_rt_time
bound = max(Run_Schedule.PickupEnd.loc[comeback1], Run_Schedule.DropoffEnd.loc[comeback1])

#is the next time window broken?
print(newETA < bound)
leftover = bound - newETA
lag1 = newETA - Run_Schedule.ETA.loc[comeback1]

#count number of broken time windows for rest of trip:
#to be able to count broken windows, amt by which they're broken,
#insert_score will contain (0) TW broken yes/no; (1) amount by which window broken
pickup_score = np.zeros(((Run_Schedule.index.max() - comeback1 + 1),2)) 
row_ctr = 0
for k in range(comeback1,(Run_Schedule.index.max()+1)):
    if (Run_Schedule.Activity.loc[k] not in [0,1]):
        row_ctr += 1
    else:
        bound = max(Run_Schedule.PickupEnd.loc[k], Run_Schedule.DropoffEnd.loc[k])
        eta_future = Run_Schedule.ETA.loc[k] + lag1
        #0 indicates TW not broken, 1 otherwise.
        pickup_score[row_ctr, 0] = int(eta_future > bound)
        #if time window is broken, by how much?
        pickup_score[row_ctr, 1] = max(0, eta_future - bound)
        #print(pickup_score[row_ctr,:])
        row_ctr+=1


#FEASIBILITY OF DROPOFF:
Run_Schedule_Lag = Run_Schedule.copy()
ETAlag = Run_Schedule.ETA + lag1
Run_Schedule_Lag.ETA = ETAlag
dropoff_inserts = time_overlap(Run_Schedule_Lag, URID, pickUpDropOff = False)
dropoff_all_nodes = filter(lambda x: x >= comeback1, dropoff_inserts["all_nodes"])
dropoff_outbound = filter(lambda x: x >= comeback1, dropoff_inserts["outbound"])
# can't return to first outbound node:
dropoff_inbound = filter(lambda x: x > comeback1, dropoff_inserts["inbound"])
if dropoff_outbound[0] == dropoff_inbound[0]: dropoff_inbound.pop(0)

outbound = Run_Schedule_Lag.loc[dropoff_outbound]
outbound = np.column_stack((np.array(outbound.LAT), np.array(outbound.LON)))
inbound = Run_Schedule_Lag.loc[dropoff_inbound]
inbound = np.column_stack((np.array(inbound.LAT), np.array(inbound.LON)))

uridLoc = [URID.DropOffCoords[0], URID.DropOffCoords[1]]
#second iteration of distance matrix, for drop off routing:
time_matrix_dropoff = osrm(uridLoc, inbound, outbound)

#start picking best pickup insertion:
rt_times = sorted(enumerate(np.sum(time_matrix_dropoff, 1)), key=operator.itemgetter(1)) #use itemgetter(1) because (0) is index from enumerator!

#smallest round trip travel time, corresponding rows on bus's schedule:
best_rt_time = rt_times[0][1]
#rows on Run_Schedule between which to insert:
leave2 = dropoff_outbound[rt_times[0][0]] #leave this scheduled node
comeback2 = dropoff_inbound[rt_times[0][0]] #come back to this scheduled node

#get total lag time, see if next time window is broken:
newETA = Run_Schedule_Lag.ETA.loc[leave2] + dwell + best_rt_time
bound = max(Run_Schedule_Lag.PickupEnd.loc[comeback2], Run_Schedule_Lag.DropoffEnd.loc[comeback2])
#total lag: lag from pickup, and then difference between lagged eta and eta for coming back from pickup
total_lag = newETA - Run_Schedule_Lag.ETA.loc[comeback2] + lag1

#count number of broken time windows from dropping off URID:
dropoff_score = np.zeros(((Run_Schedule_Lag.index.max() - comeback2 + 1),2)) 
row_ctr = 0
for k in range(comeback2,(Run_Schedule_Lag.index.max()+1)):
    if (Run_Schedule.Activity.loc[k] not in [0,1]):
        row_ctr += 1
    else:
        bound = max(Run_Schedule_Lag.PickupEnd.loc[k], Run_Schedule_Lag.DropoffEnd.loc[k])
        eta_future = Run_Schedule_Lag.ETA.loc[k] + total_lag
        #0 indicates TW not broken, 1 otherwise.
        dropoff_score[row_ctr, 0] = int(eta_future > bound)
        #if time window is broken, by how much?
        dropoff_score[row_ctr, 1] = max(0, eta_future - bound)
        #print(dropoff_score[row_ctr,:])
        row_ctr+=1

#assemble output:
pickup_df = pd.DataFrame({"nodes": range(comeback1,Run_Schedule.index.max()+1), "break_TW": pickup_score[:,0], "late": pickup_score[:,1]})
dropoff_df = pd.DataFrame({"nodes": range(comeback2,Run_Schedule.index.max()+1), "break_TW": dropoff_score[:,0], "late": dropoff_score[:,1]})
test = pickup_df[(pickup_df['nodes'] >= comeback1) & (pickup_df['nodes'] < comeback2)]
ret = {"score": test.append(dropoff_df), "pickup_insert":(leave1, comeback1), "dropoff_insert":(leave2, comeback2),
           "total_lag" : total_lag, 'RunID' : Run_Schedule.Run.iloc[0]}
           


Need to service URID within 50400 sec to 52200 sec
...6 nodes fall within this criteria.
True
Need to service URID within 46897 sec to 54097 sec
...11 nodes fall within this criteria.


In [74]:
#Score for inserting URID on to this Run_Schedule:
ret
data.loc[ret['dropoff_insert'][0]:ret['dropoff_insert'][1]]

,ServiceDate,Run,ProviderId,EvOrder,EvId,ReqTime,SchTime,ReqLate,Activity,ETA,...,PassOn,SpaceOn,PassOff,SpaceOff,ClientId,MobAids,PickupStart,PickupEnd,DropoffStart,DropoffEnd
516,2014-04-14,502R,5,56136252,13679,-1,NaN,-1,1,54606,...,NaN,NaN,CLI1,WX1,92953,AP,0,0,51006,58206
517,2014-04-14,502R,5,61475448,15017,51300,54900,-1,0,56888,...,CLI1,WG1,NaN,NaN,32104,WA,54000,55800,0,0


#How late are the buses usually?

In [111]:
def originalLateness(Run_Schedule):
    '''
    Run_Schedule: Pandas dataframe containing Trapeze-scheduled bus route and time windows, Run is listed in good_buses
    URID: of class URID

    return: pandas dataframe with 2 columns: 'break_TW' (binary variable
        indicating whether future stop will be late), 'late' (integer indicating how late bus will be to stop).
    '''

    lateness_score = np.zeros((Run_Schedule.shape[0],2))
    row_ctr = 0
    #Don't count lateness when the bus is leaving garage.
    for k in range(Run_Schedule.index.min(),(Run_Schedule.index.max()+1)):
        if (Run_Schedule.Activity.loc[k] not in [0,1]):
            row_ctr += 1
        else:
            bound = max(Run_Schedule.PickupEnd.loc[k], Run_Schedule.DropoffEnd.loc[k])
            eta = Run_Schedule.ETA.loc[k]
            #0 indicates TW not broken, 1 otherwise.
            lateness_score[row_ctr, 0] = int(eta > bound)
            #if time window is broken, by how much?
            lateness_score[row_ctr, 1] = max(0, eta - bound)
            #print(dropoff_score[row_ctr,:])
            row_ctr+=1

    ret = pd.DataFrame({"break_TW": lateness_score[:,0], "late": lateness_score[:,1]})

    return ret

late_dict = {}
for bus in good_buses:
    run_sched = get_busRuns(data, bus, resched_init_time)
    score = originalLateness(run_sched)
    late_dict[bus] = sum(score["late"])

print("Average lateness of %s runs over day:\n%s seconds" % (len(good_buses), sum(late_dict.values())/len(late_dict.values())))

Getting remaing rides for run 601R
Getting remaing rides for run 5043TRC
Getting remaing rides for run 5038
Getting remaing rides for run 5039
Getting remaing rides for run 5037
Getting remaing rides for run 5032
Getting remaing rides for run 5030
Getting remaing rides for run 5031
Getting remaing rides for run 7018
Getting remaing rides for run 4021
Getting remaing rides for run 7016
Getting remaing rides for run 4020
Getting remaing rides for run 7015
Getting remaing rides for run 7012
Getting remaing rides for run 7013
Getting remaing rides for run 7010
Getting remaing rides for run 688DEB
Getting remaing rides for run 590TRN
Getting remaing rides for run 673SEA
Getting remaing rides for run 687DEB
Getting remaing rides for run 502R
Getting remaing rides for run 6040
Getting remaing rides for run 6043
Getting remaing rides for run 570SHO
Getting remaing rides for run 6046
Getting remaing rides for run 6047
Getting remaing rides for run 368
Getting remaing rides for run 325
Getting r